In [21]:
# Notebook to create queries to the new Dexcom API to create Diabetes Ring

In [22]:
import requests
import json
import ssl
import http.client
import datetime
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import rauth

from webbrowser import open_new
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.request import urlopen, HTTPError

In [23]:
class HTTPServerHandler(BaseHTTPRequestHandler):

    def __init__(self, request, address, server):
        super().__init__(request, address, server)

    def do_GET(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
        if 'code' in self.path:
            self.wfile.write(bytes('<html><h1>You may now close this window.'
                              + '</h1></html>', 'utf-8'))
            self.server.auth_code = self.path.split('=')[1]

In [24]:
class Golden_Ticket():
    
    def __init__(self, client_id, client_sec):
        self.client_id = client_id
        self.client_sec = client_sec
        self.redirect_uri = 'http://0.0.0.0:8000/'
        
    def Assign_Ids(self, client_id, client_sec):
        self.client_id = client_id
        self.client_sec = client_sec
        
    def Get_Auth_Code(self):
        
        # Authorize with Dexcom and return Authorization Code
        url = 'https://uam1.dexcom.com/identity/connect/authorize?client_id='+ self.client_id + '&redirect_uri=' + self.redirect_uri + '&response_type=code&scope=egv%20calibration%20device%20statistics%20event%20offline_access'

        open_new(url)

        httpServer = HTTPServer(('localhost', 8000), HTTPServerHandler)

        httpServer.handle_request()
        self.auth_code = httpServer.auth_code
        httpServer.server_close()
        
    def Get_Token(self):
            
        conn = http.client.HTTPSConnection("api.dexcom.com")

        payload = "client_secret="+ self.client_sec + "&client_id="+ self.client_id + "&code="+self.auth_code+"&grant_type=authorization_code&redirect_uri="+self.redirect_uri

        headers = {
            'content-type': "application/x-www-form-urlencoded",
            'cache-control': "no-cache"
            }

        conn.request("POST", "/v1/oauth2/token", payload, headers)

        res = conn.getresponse()
        data = res.read()
        Data = json.loads(data)

        self.access_token = Data['access_token']
        self.refresh_token = Data['refresh_token']
        
    def Get_Refresh(self):
        
        conn = http.client.HTTPSConnection("api.dexcom.com")

        payload = "client_secret="+self.client_sec+"&client_id="+self.client_id+"&refresh_token="+self.refresh_token+"&grant_type=refresh_token&redirect_uri="+self.redirect_uri

        headers = {
            'content-type': "application/x-www-form-urlencoded",
            'cache-control': "no-cache"
            }

        conn.request("POST", "/v1/oauth2/token", payload, headers)

        res = conn.getresponse()
        data = res.read()
        Data = json.loads(data)

        self.access_token = Data['access_token']
        self.refresh_token = Data['refresh_token']

In [25]:
def GetAvgGlucose(StartTime,EndTime,Access_Token):
    conn = http.client.HTTPSConnection("api.dexcom.com")

    headers = {
        'authorization': "Bearer " + Access_Token
        }

    conn.request("GET", "/v1/users/self/egvs?startDate="+ StartTime +"&endDate="+EndTime, headers=headers)

    res = conn.getresponse()
    data = res.read()
    Data = json.loads(data)
    return Data

In [ ]:
# Read in Client ID and Client Secret
client_id_loc = '/Users/colinreid/Desktop/Client_ID.txt'
client_secret_loc = '/Users/colinreid/Desktop/Client_Secret.txt'

with open(client_id_loc, 'r') as myfile:
        client_id = myfile.read().replace('\n', '')

with open(client_secret_loc, 'r') as myfile:
    client_sec = myfile.read().replace('\n', '')

# Initialize Token
Ticket = Golden_Ticket(client_id,client_sec)

# Get Authorization Code
Ticket.Get_Auth_Code()

# Get Authorization Token
Ticket.Get_Token()

In [ ]:
# Run again if you need to refresh your token
Ticket.Get_Refresh()

In [ ]:
# Grab previous X days of data
Today = datetime.date.today()
Minus = Today - datetime.timedelta(days=90)

StartTime = Minus.isoformat() + 'T00:00:00'
EndTime = Today.isoformat() + 'T00:00:00'

Results = GetAvgGlucose(StartTime, EndTime, Ticket.access_token)

In [ ]:
# Print out results in a pandas dataframe
Glucose_Data = pd.DataFrame(Results['egvs'])

In [ ]:
# Calculate corresponding A1C
average_glucose = Glucose_Data['value'].mean()
A1C = (average_glucose + 46.7) / 28.7

In [ ]:
# Plot Data 

In [ ]:
# Extract data from dates for plotting purposes
%matplotlib inline
import seaborn as sns
Date = Glucose_Data['displayTime'][2]
Days = []
Hour = []
Min = []
hours = []
for i in range(Glucose_Data.shape[0]):
    Days.append(datetime.datetime.strptime(Glucose_Data['displayTime'][i], "%Y-%m-%dT%H:%M:%S").strftime('%a'))
    Hours = datetime.datetime.strptime(Glucose_Data['displayTime'][i], "%Y-%m-%dT%H:%M:%S").strftime('%H')
    hours.append(int(Hours))
    Mins = datetime.datetime.strptime(Glucose_Data['displayTime'][i], "%Y-%m-%dT%H:%M:%S").strftime('%M')
    Min.append(int(Mins) + int(Hours)*60)
    if int(Hours) > 11:
        Hour.append('PM')
    else:
        Hour.append('AM')

Glucose_Data['value'].apply(int)
Glucose_Data['Day'] = pd.Series(Days)
Glucose_Data['AM_PM'] = pd.Series(Hour)
Glucose_Data['Hours'] = pd.Series(hours)
Glucose_Data['Mins'] = pd.Series(Min) 

In [ ]:
# Plot the glucose value over time for each day
Trend_Dataset = Glucose_Data[['value','Day','Hours']]

Grouped = Trend_Dataset.groupby(['Day','Hours']).mean()
Flattened = pd.DataFrame(Grouped.to_records())
Final = Flattened.pivot("Hours", "Day", "value")
Final = Final.reindex(columns=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

In [ ]:
%matplotlib inline
start_radius = 1.5
Charts = []
    
group_names= [str(elem) + ':00' for elem in list(Final.index)]
group_size=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

# Create colors

Colors = []
for Day in ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']:
    colors = []
    for data in list(Final[Day]):
        if data > 220 or data < 80:
            colors.append('red')
        else:
            colors.append('green')
    Colors.append(colors)
    
# First Ring (outside)
fig, ax = plt.subplots()
ax.axis('equal')
fig.text(x=0,y=0,s='Weekly Diabetes Ring',size='12')
fig.text(0.5, 0.5, '{0:.2f}'.format(A1C) , horizontalalignment='center',verticalalignment='center',size =36)
mypie, _ = ax.pie(group_size, radius=start_radius, labels=group_names, labeldistance=1.05, colors=Colors[0],counterclock=False,startangle=90)
plt.setp( mypie, width=0.2, edgecolor='white')

mypie2, _ = ax.pie(group_size, radius=1.4,colors=Colors[1],counterclock=False,startangle=90)
plt.setp( mypie2, width=0.1, edgecolor='white')

mypie3, _ = ax.pie(group_size, radius=1.3, colors=Colors[2],counterclock=False,startangle=90)
plt.setp( mypie3, width=0.1, edgecolor='white')

mypie4, _ = ax.pie(group_size, radius=1.2, colors=Colors[3],counterclock=False,startangle=90)
plt.setp( mypie4, width=0.1, edgecolor='white')

mypie5, _ = ax.pie(group_size, radius=1.1, colors=Colors[4],counterclock=False,startangle=90)
plt.setp( mypie5, width=0.1, edgecolor='white')

mypie6, _ = ax.pie(group_size, radius=1.0, colors=Colors[5],counterclock=False,startangle=90)
plt.setp( mypie6, width=0.1, edgecolor='white')

mypie7, _ = ax.pie(group_size, radius=0.9, colors=Colors[6],counterclock=False,startangle=90)
plt.setp( mypie7, width=0.1, edgecolor='white')

mypie7, _ = ax.pie(group_size, radius=0.9, colors=Colors[6],counterclock=False,startangle=90)
plt.setp( mypie7, width=0.1, edgecolor='white')

fig.set_size_inches(7,7)

plt.margins(0,0)
    

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = [16,9]

Day = 'Mon'

# set up random data between 0 and 90
r = list(Final[Day])
r.append(r[0])

# set up 24 hours matching the random data above
hours = np.linspace(0.0,24.0,len(r))

# scaling the 24 hours to the full circle, 2pi
theta = hours / 24.0 * (2.0 * np.pi)

# reverse your data, so that 90 becomes 0:
r_rev = [(ri - 400) * -1.0 for ri in r]

# set up your polar plot
ax = plt.subplot(111, projection='polar')
ax.plot(theta, r, color='r', linewidth=3)

# define your axis limits
ax.set_ylim([0, 400.0])

ax.set_yticklabels(['{:.0f}'.format(ylabel) \
                for ylabel in np.linspace(0.0,400.0,len(ax.get_yticklabels())+1)[1:]])


ax.set_xticklabels(['{:.0f}:00'.format(xlabel) \
                    for xlabel in np.arange(0.0,24.0,(24.0 / len(ax.get_xticklabels())))])

ax.grid(True)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)


plt.fill_between(theta, 80, 200, facecolor='green', alpha=0.2)
plt.fill_between(theta, 60, 80, facecolor='yellow', alpha=0.2)
plt.fill_between(theta, 200, 250, facecolor='yellow', alpha=0.2)
plt.fill_between(theta, 0, 60, facecolor='red', alpha=0.1)
plt.fill_between(theta, 250, 400, facecolor='red', alpha=0.1)

plt.title(s=Day + 'day Glucose Circle')
plt.show()


In [ ]:
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = [16,9]

Day = 'Sat'

# set up random data between 0 and 90
r = list(Final[Day])
r.append(r[0])

r_above = [data if data > 200 else None for data in r]
r_below = [data if data < 70 else None for data in r]
r_middle = [data if data >= 70 and data <=200 else None for data in r]

# set up 24 hours matching the random data above
hours = np.linspace(0.0,24.0,len(r))

# scaling the 24 hours to the full circle, 2pi
theta = hours / 24.0 * (2.0 * np.pi)

# reverse your data, so that 90 becomes 0:
r_rev = [(ri - 400) * -1.0 for ri in r]

# set up your polar plot
ax = plt.subplot(111, projection='polar')
ax.plot(theta, r, color='black', linewidth=2,linestyle='dashed')
#ax.plot(theta, r, color='r', linewidth=3)
# define your axis limits
ax.set_ylim([0, 400.0])

ax.set_yticklabels(['{:.0f}'.format(ylabel) \
                for ylabel in np.linspace(0.0,400.0,len(ax.get_yticklabels())+1)[1:]])


ax.set_xticklabels(['{:.0f}:00'.format(xlabel) \
                    for xlabel in np.arange(0.0,24.0,(24.0 / len(ax.get_xticklabels())))])

ax.grid(True)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)

Above200 = [True if value > 200 else False for value in r]
Below60 = [True if value < 70 else False for value in r]

plt.fill_between(theta, 200, r , where=Above200, interpolate=True, facecolor='red', alpha=0.5)
plt.fill_between(theta, r, 70 , where=Below60, interpolate=True, facecolor='red', alpha=0.5)
plt.fill_between(theta, 70, 200, facecolor='green', alpha=0.2)


plt.title(s=Day + 'day Glucose Circle')
plt.show()